In [16]:
!playwright codegen

^C


In [17]:
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import pandas as pd

# Placeholder for all data
all_data = []

# Start Playwright and browser
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
context = await browser.new_context()
page = await context.new_page()
await page.goto("https://jp.mercari.com/search?category_id=79")

while True:
    # Scrape the current page
    html = await page.content()
    soup = BeautifulSoup(html, 'html.parser')
    
    # Locate all item listings by the `data-testid` attribute
    items = soup.find_all('li', {'data-testid': 'item-cell'})
    
    for item in items:
        # Extract relevant data fields
        url_tag = item.find('a', {'data-testid': 'thumbnail-link'})
        price_tag = item.find('span', {'class': 'number__6b270ca7'})
        img_tag = item.find('img')
        name_tag = item.find('div', {'class': 'imageContainer__f8ddf3a2'})
        
        data = {
            'url': url_tag['href'] if url_tag else None,
            'price': price_tag.text if price_tag else None,
            'thumbnail': img_tag['src'] if img_tag else None,
            'name': name_tag['aria-label'] if name_tag else None
        }
        
        # Add the extracted data to the list
        all_data.append(data)
    
    # Try to click the "Next" button
    try:
        await page.locator("a:has-text('次へ')").click(timeout=10000)
    except:
        # If the button is not found, end the loop
        break

# Close browser
await context.close()
await browser.close()

# Convert collected data to a pandas DataFrame and display
df = pd.DataFrame(all_data)
print(df.head())  # This will display the first few rows of the DataFrame


Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed\nCall log:\nwaiting for locator("a:has-text(\'次へ\')")\n')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
Call log:
waiting for locator("a:has-text('次へ')")



ERROR IS Connection closed while reading from the driver


Exception: Browser.close: Connection closed while reading from the driver

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

# Initialize an empty list to store product data
all_data = []

async def scrape_page(page):
    # Wait for the page content to load
    await page.wait_for_selector("li[data-testid='item-cell']", timeout=10000)

    # Get the HTML content of the page
    content = await page.content()

    # Parse the page using BeautifulSoup
    soup = BeautifulSoup(content, 'html.parser')

    # Find all product listing elements
    items = soup.find_all("li", {"data-testid": "item-cell"})

    for item in items:
        try:
            # Extract product details with proper checks for missing elements
            product_url = item.find('a')['href'] if item.find('a') else None
            if product_url:
                product_url = "https://jp.mercari.com" + product_url
            
            product_name = item.find('div', {'role': 'img'})['aria-label'] if item.find('div', {'role': 'img'}) else None
            price = item.find('span', {'class': 'number__6b270ca7'}).text if item.find('span', {'class': 'number__6b270ca7'}) else None
            image_url = item.find('img')['src'] if item.find('img') else None

        except AttributeError:
            # Handle any unexpected attribute errors
            product_url = product_name = price = image_url = None

        # Add data to the list as a dictionary
        all_data.append({
            'Product URL': product_url,
            'Product Name': product_name,
            'Price': price,
            'Image URL': image_url
        })

# Start Playwright and run the scraping process
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
context = await browser.new_context()
page = await context.new_page()

# Go to the initial page
await page.goto("https://jp.mercari.com/search?category_id=79", timeout=10000)

# Scrape data from all pages
while True:
    await scrape_page(page)
    
    # Try clicking the "Next Page" button
    try:
        await page.get_by_role("link", name="次へ").click(timeout=10000)
    except:
        # If the button is not found or there's an error, break the loop
        break

await context.close()
await browser.close()



In [15]:
# Save the data to a pandas DataFrame
df = pd.DataFrame(all_data)

# Show the first few rows to confirm the result
df.head()


,url,price,thumbnail,name
0,/item/m73950686855,"5,000",https://static.mercdn.net/c!/w=240/thumb/photo...,ピアノプレーヤ レイ・ブライアントのサムネイル
1,/item/m62887375584,"1,500",https://static.mercdn.net/c!/w=240/thumb/photo...,ブルーマングループ ドラムスティックのサムネイル
2,/item/m57288519399,"1,500",https://static.mercdn.net/c!/w=240/thumb/photo...,ハック BEAT SELECTのサムネイル
3,/item/m58807281652,???,https://static.mercdn.net/c!/w=240/thumb/photo...,レア fernandes FR カタログ外 リバースヘッド コンコルドのサムネイル
4,/item/m86880361736,"3,500",https://static.mercdn.net/c!/w=240/thumb/photo...,kk様専用のサムネイル


In [ ]:
df.to_csv("mercari.csv", index=False)

In [ ]:
import pandas as pd
from playwright.async_api import async_playwright, expect

async def run(playwright):
    browser = await playwright.chromium.launch(headless=False)
    context = await browser.new_context()
    page = await context.new_page()
    await page.goto("https://suumo.jp/jj/chintai/ichiran/FR301FC005/?sc=13101&ar=030&bs=040&sd=1&ta=13&ts=1")

    all_data = []

    while True:
        await page.wait_for_selector(".property")  # Wait for property elements to load
        
        # Get all property elements on the current page
        properties = await page.query_selector_all(".property")
        
        for property in properties:
            title = await property.query_selector("h2.property_inner-title a")
            title_text = await title.inner_text()
            link = await title.get_attribute("href")
            price = await property.query_selector(".detailbox-property-point")
            price_text = await price.inner_text() if price else "N/A"
            
            # Add more fields as needed
            
            # Append the data as a dictionary
            all_data.append({
                "title": title_text,
                "link": link,
                "price": price_text,
                # Add more fields here
            })
        
        # Try to click the "Next" button
        try:
            next_button = await page.query_selector("p.pagination-parts a:has-text('次へ')")
            if next_button:
                await next_button.click()
            else:
                break  # Exit if no next button found
        except:
            break  # Exit on any error (like timeout)
    
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(all_data)
    df.head()  # Display the first few rows
    
    await context.close()
    await browser.close()

async def main():
    async with async_playwright() as playwright:
        await run(playwright)

# Run the main function
await main()


In [22]:
import pandas as pd

df = pd.read_csv("cleaning-worksheets/suumo-properties.csv")
df.head()

,Property Name,Rent,Management Fee,Deposit,Key Money,Floor Plan,Area Size,Direction,Building Age,Address,Station Info,Agency Name,Agency Phone,Property URL
0,クラッシィハウス大手町,32万円,管理費 15000円,敷32万円,礼32万円,2LDK,54.13m2,東,NaN,NaN,\n東京メトロ半蔵門線/大手町駅 歩4分\n東京メトロ丸ノ内線/淡路町駅 歩6分\nＪＲ中央...,NaN,(株)プライムコーポレーション都心営業部,https://suumo.jp/chintai/bc_100400768898/
1,ライオンズフォーシア神田須田町,23万円,管理費 20000円,敷23万円,礼-,2LDK,45.57m2,南,NaN,NaN,\nＪＲ山手線/秋葉原駅 歩5分\n東京メトロ銀座線/神田駅 歩6分\n都営新宿線/岩本町駅...,NaN,(株)ユウキ・ホーム麻布十番網代公園店,https://suumo.jp/chintai/bc_100335153441/
2,ライオンズフォーシア神田須田町,23万円,管理費 20000円,敷23万円,礼-,2LDK,45.57m2,南,NaN,NaN,\nＪＲ山手線/秋葉原駅 歩5分\n東京メトロ銀座線/神田駅 歩6分\n都営新宿線/岩本町駅...,NaN,(株)ユウキ・ホーム麻布十番サテライト店,https://suumo.jp/chintai/bc_100335153437/
3,ライオンズフォーシア神田須田町,23万円,管理費 20000円,敷23万円,礼-,2LDK,45.57m2,南,NaN,NaN,\nＪＲ山手線/秋葉原駅 歩5分\n東京メトロ銀座線/神田駅 歩6分\n都営新宿線/岩本町駅...,NaN,(株)ユウキ・ホーム麻布十番店,https://suumo.jp/chintai/bc_100335153675/
4,ライオンズフォーシア神田須田町,23万円,管理費 20000円,敷23万円,礼-,2LDK,45.57m2,南,NaN,NaN,\nＪＲ山手線/秋葉原駅 歩5分\n東京メトロ銀座線/神田駅 歩6分\n都営新宿線/岩本町駅...,NaN,そらうみリアルエステート(株)本店,https://suumo.jp/chintai/bc_100335156273/


/Users/soma/Library/CloudStorage/Dropbox/Soma/Curriculum/2024-nikkei/03-scraping
